In [1]:
# Importing libraries and initial configs
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100, "display.max_rows", 100)


### Case Study #2

In [2]:
df = pd.read_csv("./data/casestudy.csv").iloc[:, 1:]


In [3]:
df.head()


,customer_email,net_revenue,year
0,nhknapwsbx@gmail.com,249.92,2015
1,joiuzbvcpn@gmail.com,87.61,2015
2,ukkjctepxt@gmail.com,168.38,2015
3,gykatilzrt@gmail.com,62.40,2015
4,mmsgsrtxah@gmail.com,43.08,2015


In [4]:
df.shape


(685927, 3)

In [5]:
df.duplicated(["customer_email", "year"]).any()


False

#### Total revenue for the current year

In [6]:
df.groupby("year")["net_revenue"].sum()


year
2015    29036749.19
2016    25730943.59
2017    31417495.03
Name: net_revenue, dtype: float64

#### New Customer Revenue e.g. new customers not present in previous year only

In [7]:
def calc_new_rev(prev_year, curr_year):
    """
    Calculates net revenue for new customers, not present in previous year
        Inputs:
            prev_year: Previous year
            curr_year: Current year
        Output:
            Net revenue for new customers in current year
    """
    cust_prev = set(df.loc[df["year"] == prev_year]["customer_email"])
    cust_curr = set(df.loc[df["year"] == curr_year]["customer_email"])
    new_cust = cust_curr.difference(cust_prev)
    return df.iloc[
        np.where(
            np.logical_and(df["customer_email"].isin(new_cust), df["year"] == curr_year)
        )[0],
        :,
    ]["net_revenue"].sum()


In [8]:
for (i, j) in zip(range(2015, 2017), range(2016, 2018)):
    print(
        "The net revenue from new customers in {} not present in {} is : {:.2f}".format(
            j, i, calc_new_rev(i, j)
        )
    )


The net revenue from new customers in 2016 not present in 2015 is : 18245491.01
The net revenue from new customers in 2017 not present in 2016 is : 28776235.04


#### Existing Customer Growth. To calculate this, use the Revenue of existing customers for current year –(minus) Revenue of existing customers from the previous year

In [9]:
def calc_exist_rev(curr_year, prev_year):
    """
    Calculates growth in revenue for existing customers in current year from previous year
        Inputs:
            curr_year: Current year
            prev_year: Previous year
        Output:
            Growth in revenue for existing customers in current year from previous year
    """
    cust_curr = set(df.loc[df["year"] == curr_year]["customer_email"])
    cust_prev = set(df.loc[df["year"] == prev_year]["customer_email"])
    exist_cust = cust_prev.intersection(cust_curr)

    return (
        df.iloc[
            np.where(
                np.logical_and(
                    df["customer_email"].isin(exist_cust), df["year"] == curr_year
                )
            )[0],
            :,
        ]["net_revenue"].sum()
        - df.iloc[
            np.where(
                np.logical_and(
                    df["customer_email"].isin(exist_cust), df["year"] == prev_year
                )
            )[0],
            :,
        ]["net_revenue"].sum()
    )


In [10]:
for (i, j) in zip(range(2015, 2017), range(2016, 2018)):
    print(
        "The net revenue in {} from existing customers in {} is : {:.2f}".format(
            j, i, calc_exist_rev(j, i)
        )
    )


The net revenue in 2016 from existing customers in 2015 is : 20335.46
The net revenue in 2017 from existing customers in 2016 is : 20611.34


#### Revenue lost from attrition

In [11]:
def calc_att_rev(curr_year, prev_year):
    """
    Calculates revenue lost from attrition in current year from previous year
        Inputs:
            curr_year: Current year
            prev_year: Previous year
        Output:
            Revenue lost from attrition in current year
    """
    cust_curr = set(df.loc[df["year"] == curr_year]["customer_email"])
    cust_prev = set(df.loc[df["year"] == prev_year]["customer_email"])
    exist_cust = cust_prev.difference(cust_curr)

    return df.iloc[
        np.where(
            np.logical_and(
                df["customer_email"].isin(exist_cust), df["year"] == prev_year
            )
        )[0],
        :,
    ]["net_revenue"].sum()


In [12]:
for (i, j) in zip(range(2015, 2017), range(2016, 2018)):
    print(
        "The revenue lost from attrition in {} is : {:.2f}".format(
            j, calc_att_rev(j, i)
        )
    )


The revenue lost from attrition in 2016 is : 21571632.07
The revenue lost from attrition in 2017 is : 23110294.94


#### Existing Customer Revenue Current Year

In [13]:
def calc_exist_rev_current(curr_year, prev_year):
    """
    Calculates revenue for existing customers in current year
        Inputs:
            curr_year: Current year
            prev_year: Previous year
        Output:
            Revenue for existing customers in current year
    """
    cust_curr = set(df.loc[df["year"] == curr_year]["customer_email"])
    cust_prev = set(df.loc[df["year"] == prev_year]["customer_email"])
    exist_cust = cust_prev.intersection(cust_curr)

    return df.iloc[
        np.where(
            np.logical_and(
                df["customer_email"].isin(exist_cust), df["year"] == curr_year
            )
        )[0],
        :,
    ]["net_revenue"].sum()


In [14]:
for (i, j) in zip(range(2015, 2017), range(2016, 2018)):
    print(
        "The revenue from existing customers in {} is : {:.2f}".format(
            j, calc_exist_rev_current(j, i)
        )
    )


The revenue from existing customers in 2016 is : 7485452.58
The revenue from existing customers in 2017 is : 2641259.99


#### Existing Customer Revenue Prior Year

In [15]:
def calc_exist_rev_prev(curr_year, prev_year):
    """
    Calculates revenue for existing customers in previous year
        Inputs:
            curr_year: Current year
            prev_year: Previous year
        Output:
            Revenue for existing customers in previous year
    """
    cust_curr = set(df.loc[df["year"] == curr_year]["customer_email"])
    cust_prev = set(df.loc[df["year"] == prev_year]["customer_email"])
    exist_cust = cust_prev.intersection(cust_curr)

    return df.iloc[
        np.where(
            np.logical_and(
                df["customer_email"].isin(exist_cust), df["year"] == prev_year
            )
        )[0],
        :,
    ]["net_revenue"].sum()


In [16]:
for (i, j) in zip(range(2015, 2017), range(2016, 2018)):
    print(
        "The revenue from existing customers in {} is : {:.2f}".format(
            i, calc_exist_rev_prev(j, i)
        )
    )


The revenue from existing customers in 2015 is : 7465117.12
The revenue from existing customers in 2016 is : 2620648.65


#### Total Customers Current Year

In [17]:
def calc_cust_curr(curr_year):
    """
    Calculates number of customers in current year
        Inputs:
            curr_year: Current year
    """
    cust_curr = set(df.loc[df["year"] == curr_year]["customer_email"])

    return len(cust_curr)


In [18]:
for i in [2015, 2016, 2017]:
    print("The number of customers in {} is : {:,}".format(i, calc_cust_curr(i)))


The number of customers in 2015 is : 231,294
The number of customers in 2016 is : 204,646
The number of customers in 2017 is : 249,987


#### Total Customers Previous Year

In [19]:
def calc_cust_prev(curr_year):
    """
    Calculates number of customers in previous year
        Inputs:
            curr_year: Current year
    """
    if curr_year == 2015:
        return 0
    else:
        cust_prev = set(df.loc[df["year"] == curr_year - 1]["customer_email"])

    return len(cust_prev)


In [20]:
for i in [2015, 2016, 2017]:
    if i == 2015:
        print(
            "Current year is {}, number of customers in the previous year is : {:,}".format(
                i, calc_cust_prev(i)
            )
        )
    else:
        print(
            "Current year is {}, number of customers in the previous year {} is : {:,}".format(
                i, i - 1, calc_cust_prev(i)
            )
        )


Current year is 2015, number of customers in the previous year is : 0
Current year is 2016, number of customers in the previous year 2015 is : 231,294
Current year is 2017, number of customers in the previous year 2016 is : 204,646


#### New Customers

In [21]:
def calc_new_cust(prev_year, curr_year):
    """
    Calculates new customers in current year compared to previous year
        Inputs:
            prev_year: Previous year
            curr_year: Current year
        Output:
            Net revenue for new customers in current year
    """
    cust_prev = set(df.loc[df["year"] == prev_year]["customer_email"])
    cust_curr = set(df.loc[df["year"] == curr_year]["customer_email"])
    new_cust = cust_curr.difference(cust_prev)
    return len(new_cust)


In [22]:
for (i, j) in zip(range(2015, 2017), range(2016, 2018)):
    print(
        "The number of new customers in {} compared to {} is : {:.2f}".format(
            j, i, calc_new_cust(i, j)
        )
    )


The number of new customers in 2016 compared to 2015 is : 145062.00
The number of new customers in 2017 compared to 2016 is : 229028.00


#### Lost Customers

In [23]:
def calc_att_cust(curr_year, prev_year):
    """
    Calculates number of customers attrited in current year from previous year
        Inputs:
            curr_year: Current year
            prev_year: Previous year
    """
    cust_curr = set(df.loc[df["year"] == curr_year]["customer_email"])
    cust_prev = set(df.loc[df["year"] == prev_year]["customer_email"])
    exist_cust = cust_prev.difference(cust_curr)

    return len(exist_cust)


In [24]:
for (i, j) in zip(range(2015, 2017), range(2016, 2018)):
    print(
        "The number of customers attrited in {} from {} is : {:.2f}".format(
            j, i, calc_att_cust(i, j)
        )
    )


The number of customers attrited in 2016 from 2015 is : 145062.00
The number of customers attrited in 2017 from 2016 is : 229028.00
